In [1]:
import re
from sympy.solvers import solve 
from sympy import Symbol
from itertools import combinations, product

In [2]:
%%time
solve_eq = '+ min(400, 400*a) + min(400, 400*a) + min(0, 100*a) = 200'
value = re.search(r"=\s+(\d+)", solve_eq).group(1)
compartments = re.findall(r"(\+|\-)\s+(min|max)(\([^\)]+\))", solve_eq) 
compartments

Wall time: 996 µs


[('+', 'min', '(400, 400*a)'),
 ('+', 'min', '(400, 400*a)'),
 ('+', 'min', '(0, 100*a)')]

In [4]:
m = re.findall(r"(\d+[\*a-z]*),\s+(\d+[\*a-z]*)", '(400, 400*a)')
m

[('400', '400*a')]

In [6]:
%%time
lists = [] 
for equation in compartments:
    match = re.findall(r"(\d+[\*a-z]*),\s+(\d+[\*a-z]*)", equation[2])
    lists.append(match[0])
lists

Wall time: 0 ns


[('400', '400*a'), ('400', '400*a'), ('0', '100*a')]

In [7]:
%%time
products = list(product(*lists))
products

Wall time: 0 ns


[('400', '400', '0'),
 ('400', '400', '100*a'),
 ('400', '400*a', '0'),
 ('400', '400*a', '100*a'),
 ('400*a', '400', '0'),
 ('400*a', '400', '100*a'),
 ('400*a', '400*a', '0'),
 ('400*a', '400*a', '100*a')]

In [8]:
def convert_letters(matchobj):
    m = matchobj.group()
    match_map = {m: "*result[0]"} if len(m) == 2 else {m: m}
    return match_map[m]

In [10]:
a_replace = re.sub(r"\*([a-z]+)", convert_letters, solve_eq)
a_replace = a_replace.replace(r"=", "==") 
a = Symbol("a")
sols = [] 

for p in products:
    knit = ""
    for i, operand in enumerate(p):
        operator = compartments[i][0]
        knit += f"{operator}{operand}"
    knit += f"-{value}"
    result = solve(eval(knit), a)
    if len(result) == 0:
        continue
    if eval(a_replace):
        print(result[0])

1/4
